In [1]:
import numpy as np
import sys
import pathlib
from os.path import join
path_to_file = str(pathlib.Path().resolve())
dir_path = join(path_to_file)

sys.path.append(join(dir_path, "HelperFiles"))

# import importlib
import retrospective
import helper
import top_k
import train_models
import load_data
from os.path import join
data_path = join(dir_path, "Experiments", "Data")

import warnings
warnings.filterwarnings('ignore')

## Load dataset and train model

Git repo has built-in "bank", "brca", "census" (AKA adult), and "credit" datasets.
Can fit logistic regression, random forest, or neural net.

In [ ]:
dataset = "census"
X_train, y_train, X_test, y_test, mapping_dict = load_data.load_data(data_path, dataset)
model = train_models.train_model(X_train, y_train, "nn", lime=False)

Y_preds = (model(X_test) > 0.5).astype("int")
print("NN {}% accuracy".format(round(np.mean(Y_preds == y_test)*100)))
d = len(mapping_dict) if mapping_dict is not None else X_train.shape[1]
print(d)

NN 82% accuracy
12


# Retrospective Analysis
## Shapley Sampling 

In [101]:
np.random.seed(1)
x_idx = 1
xloc = X_test[x_idx]
ss_vals, ss_vars = retrospective.shapley_sampling(model, X_train, xloc, n_perms=5000//d, n_samples_per_perm=10, 
                                                  mapping_dict=mapping_dict, abs=True)

### Number of verified ranks

In [102]:
alphas = [0.2, 0.1, 0.05, 0.01]
print("Alphas: ", alphas)
n_verif = [helper.find_num_verified(ss_vals, ss_vars, alpha=alpha, abs=True) for alpha in alphas]
print("Number of verified ranks with Shapley Sampling:", n_verif)

Alphas:  [0.2, 0.1, 0.05, 0.01]
Number of verified ranks with Shapley Sampling: [5, 5, 5, 5]


### Top K set

In [103]:
for K in [5,6]:
    alpha = 0.05
    result = helper.test_top_k_set(ss_vals, ss_vars, K=K, alpha=alpha, abs=True,
                                compute_sample_size=True, value_vars=ss_vars*(5000//d), 
                                n_equal=True, return_p_val=True, return_close_ranks=True)
    test_result, pair_idx, n_to_reject_pair, p_val, rank_idx = result
    print(f"Top-{K} Set Stability at alpha={alpha}: {test_result} (p={round(p_val,4)})")
    if test_result=="fail to reject":
        print("Ranks that failed to reject: ", rank_idx)
        print("Estimated number of samples to reject pair: ", n_to_reject_pair)


Top-5 Set Stability at alpha=0.05: reject (p=0.0008)
Top-6 Set Stability at alpha=0.05: fail to reject (p=0.6974)
Ranks that failed to reject:  [6 7]
Estimated number of samples to reject pair:  [10571. 10571.]


## KernelSHAP
### Number of verified ranks

In [104]:
np.random.seed(42)
x_idx = 3
xloc = X_test[x_idx]
kshap_vals, kshap_covs = retrospective.kernelshap(model, X_train, xloc, n_perms=5000, n_samples_per_perm=10, 
                                                  mapping_dict=mapping_dict, abs=True)
kshap_vars = np.diag(kshap_covs)
n_verif_kshap = [helper.find_num_verified(kshap_vals, kshap_vars, alpha=alpha, abs=True) for alpha in [0.2, 0.1, 0.05, 0.01]]

print(f"Number of verified ranks with Shapley Sampling at alpha={alphas}: {n_verif_kshap}")



Number of verified ranks with Shapley Sampling at alpha=[0.2, 0.1, 0.05, 0.01]: [3, 3, 3, 3]


In [105]:
for K in [3,4]:
    alpha = 0.05

    test_result, p_val, rank_idx = helper.test_top_k_set(kshap_vals, kshap_vars, K=K, alpha=alpha, abs=True,
                                return_p_val=True, return_close_ranks=True)
    print(f"Top-{K} Set Stability at alpha={alpha}: {test_result} (p={round(p_val,4)})")
    if test_result=="fail to reject":
        print("Ranks that failed to reject: ", rank_idx)


Top-3 Set Stability at alpha=0.05: reject (p=0.0)
Top-4 Set Stability at alpha=0.05: fail to reject (p=0.3213)
Ranks that failed to reject:  [4 5]


# Top-K methods
## RankSHAP

In [106]:
np.random.seed(123)
K = 5
xloc = X_test[6]
rankshap_vals, diffs, N, converged = top_k.rankshap(model, X_train, xloc, K=K, alpha=0.1, 
                                        mapping_dict=mapping_dict, max_n_perms=10000, n_equal=True,
                                        abs=True, n_init=100)

print("Converged: {}".format(converged))
ranking = helper.get_ranking(rankshap_vals, abs=True)
print("Top-K Feature Ranking: {}".format(ranking[:K]))
n_perms_per_feature = [len(diffs[j]) for j in range(d)]
print("Total number of perms: ", N)
print("Number of perms per feature (sorted): {}".format(np.sort(n_perms_per_feature)[::-1]))


Converged: True
Top-K Feature Ranking: [3 0 7 2 5]
Total number of perms:  14186
Number of perms per feature (sorted): [3808 1599 1599  795  795  100  100  100  100  100  100  100]


## SPRT-SHAP

In [107]:
np.random.seed(42)
K = 2
xloc = X_test[1]
kshap_vals, kshap_covs, N_total, converged = top_k.sprtshap(model, X_train, xloc, K, 
                                                      mapping_dict=mapping_dict, guarantee="rank",
                                                      n_samples_per_perm=10, n_perms_btwn_tests=1000, n_max=50000, 
                                                      alpha=0.1, beta=0.2, abs=True)
print("Converged: {}".format(converged))
ranking = helper.get_ranking(kshap_vals, abs=True)
print("Top-K Feature Ranking: {}".format(ranking[:K]))


Converged: True
Top-K Feature Ranking: [3 2]
